In [101]:
# Death rate by ages from
# https://www.buzzfeednews.com/article/lamvo/coronavirus-death-rates-age-charts-us-china

import numpy as np
death_rate = np.flip(np.repeat([14.8,14.8,14.8,8.0,3.6,1.3,0.4,0.2,0.2,0.2,0.0],2)[1:])/100

# Age pyramids from
# https://www.populationpyramid.net/italy/2019/
    
import pandas as pd
kor = pd.read_csv('Republic of Korea-2019.csv', delimiter = ',')
itl = pd.read_csv('Italy-2019.csv', delimiter = ',')
for df in [kor,itl]:
    df['total'] = df.M+df.F
    df['ratio'] = df.total / df.total.sum()*100

kor_itl = pd.DataFrame( [kor.Age, kor.ratio, itl.ratio, kor.ratio - itl.ratio]).transpose()
kor_itl.columns = ['Age','Korea', 'Italy', 'K - I']
kor_itl['death_rate'] = pd.Series(death_rate) 

kor_itl['death_rate_korea'] = kor_itl.Korea * kor_itl.death_rate 
kor_itl['death_rate_italy'] = kor_itl.Italy * kor_itl.death_rate
total = kor_itl[10:].sum()
total.Age = '50+'
kor_itl.loc["50+"] =total 
# kor_itl.loc["50+"].Age = "50+"
pd.options.display.float_format = '{:,.2f}'.format

kor_itl

,Age,Korea,Italy,K - I,death_rate,death_rate_korea,death_rate_italy
0,0-4,3.84,3.93,-0.09,0.00,0.00,0.00
1,5-9,4.43,4.49,-0.06,0.00,0.00,0.00
2,10-14,4.47,4.74,-0.27,0.00,0.01,0.01
3,15-19,5.07,4.74,0.33,0.00,0.01,0.01
4,20-24,6.49,4.88,1.61,0.00,0.01,0.01
5,25-29,6.78,5.25,1.53,0.00,0.01,0.01
6,30-34,6.52,5.60,0.92,0.00,0.01,0.01
7,35-39,7.53,6.13,1.40,0.00,0.02,0.01
8,40-44,7.79,7.14,0.65,0.00,0.03,0.03
9,45-49,8.47,8.09,0.38,0.00,0.03,0.03


In [117]:
# Data from https://www.worldometers.info/coronavirus/countries-where-coronavirus-has-spread/
countries = pd.read_csv('corona_cases_countries_20_03_14.csv', delimiter = ',')
countries['death_per_case'] = countries['Total Deaths'] / countries['Total Cases']*100
countries[['Country/Other','Total Cases','death_per_case']][:12]

,Country/Other,Total Cases,death_per_case
0,China,"80,824.00",3.95
1,Italy,"21,157.00",6.81
2,Iran,"12,729.00",4.80
3,S. Korea,"8,086.00",0.89
4,Spain,"6,315.00",3.06
5,Germany,"4,525.00",0.18
6,France,"4,469.00",2.04
7,USA,"2,499.00",2.20
8,Switzerland,"1,375.00",0.95
9,UK,"1,140.00",1.84
